In [221]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
pd.options.display.float_format = '{:.2f}'.format


# 예약id, 차량id 컬럼 및 예약생성시 위치좌표(결측치 3/4가량)컬럼 drop
df = pd.read_csv('../data/socar_reservation_triplog.csv', parse_dates=['reservation_start_at', 'reservation_return_at', 'member_created_date'])
df = df.drop(columns=['reservation_id', 'car_id', 'reservation_created_lat', 'reservation_created_lng'])

In [222]:
df.head()

,member_id_encrypted,region,reservation_return_at,reservation_start_at,member_age,member_gender,member_created_date,member_total_distance,is_vroom,car_name,zone_name,zone_address,zone_lat,zone_lng,zone_type1,zone_type2,zone_type3,trip
0,6qA1i9+DXTJYgfWPqSGhszRpgNkUzFATBTG4XG7OciA=,서울특별시 강서구,2019-01-01 17:33:42,2018-12-31 17:30:00,48,male,2016-12-10,2277.00,False,아반떼AD,SK주차장,서울 강서구 방화동 500-8,37.57,126.80,LIVING_APT,TRANSFER_STATION,TRANSFER_SUBWAY,NaN
1,9uzg+C7Ng5hsI0So4WgJaUME+1dgR1LXyy4c8TEM1zY=,울산광역시 남구,2019-01-01 12:08:55,2018-12-31 16:00:00,25,male,2015-07-06,13156.00,False,스포티지,우정유료주차장,울산 남구 삼산동 1481-10,35.54,129.34,TRANSFER_TERMINAL,TRANSFER_TERMINAL,COMMERCIAL_HOTSPOT,NaN
2,SwEY5le8plfdi3EKoLBAF87zCTzFgIUo4nGWkZ6tOcM=,경기도 김포시,2019-01-02 17:30:00,2019-01-02 14:50:00,34,male,2015-10-07,318.00,True,카니발 11인승,월드애비뉴,경기 김포시 구래동 6880-7,37.65,126.63,LIVING_APT,COMMERCIAL_HOTSPOT,ETC,NaN
3,vmk4kp/a2AYP8VnGWJUYlbSOff+SxEG18D6fqKRkAOk=,경기도 남양주시,2019-01-01 20:53:09,2018-12-31 11:00:00,23,male,2017-10-13,1288.00,False,아반떼AD,제뉴빌딩주차장,경기 남양주시 호평동 642-2,37.65,127.24,LIVING_APT,TRANSFER_SUBWAY,COMMERCIAL_HOTSPOT,NaN
4,gdP+78Hhm+MZ4ZxIm8kNVvMNzKJBxbiDfBsflsMcwLA=,서울특별시 강서구,2019-01-04 14:06:03,2019-01-03 08:00:00,35,male,2017-08-13,2658.00,False,투싼(경유),용정주차장,서울 강서구 방화동 608-9,37.57,126.82,LIVING_ETC,TRANSFER_SUBWAY,ETC,NaN


In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751548 entries, 0 to 751547
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   member_id_encrypted    751548 non-null  object        
 1   region                 751548 non-null  object        
 2   reservation_return_at  751548 non-null  datetime64[ns]
 3   reservation_start_at   751548 non-null  datetime64[ns]
 4   member_age             751548 non-null  int64         
 5   member_gender          699017 non-null  object        
 6   member_created_date    751548 non-null  datetime64[ns]
 7   member_total_distance  738010 non-null  float64       
 8   is_vroom               751548 non-null  bool          
 9   car_name               751548 non-null  object        
 10  zone_name              750774 non-null  object        
 11  zone_address           751548 non-null  object        
 12  zone_lat               751548 non-null  floa

In [224]:
def preprocess(df):
    # 활용할 컬럼의 null 행 제거
    COLS = ['member_age', 'member_gender', 'trip']
    df = df.dropna(subset=COLS)

    df = df.rename(columns={'member_id_encrypted':'member_id'})
    return df.copy()


In [225]:
df = preprocess(df)

In [226]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 516650 entries, 5735 to 751547
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   member_id              516650 non-null  object        
 1   region                 516650 non-null  object        
 2   reservation_return_at  516650 non-null  datetime64[ns]
 3   reservation_start_at   516650 non-null  datetime64[ns]
 4   member_age             516650 non-null  int64         
 5   member_gender          516650 non-null  object        
 6   member_created_date    516650 non-null  datetime64[ns]
 7   member_total_distance  509060 non-null  float64       
 8   is_vroom               516650 non-null  bool          
 9   car_name               516650 non-null  object        
 10  zone_name              516068 non-null  object        
 11  zone_address           516650 non-null  object        
 12  zone_lat               516650 non-null  f

In [227]:
code = pd.read_csv('../data/region_data/행정구역코드_sgis_201906.csv')
code.head()

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭
0,11,서울특별시,11010,종로구,1101053,사직동
1,11,서울특별시,11010,종로구,1101054,삼청동
2,11,서울특별시,11010,종로구,1101055,부암동
3,11,서울특별시,11010,종로구,1101056,평창동
4,11,서울특별시,11010,종로구,1101057,무악동


In [228]:
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3512 entries, 0 to 3511
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도코드    3512 non-null   int64 
 1   시도명칭    3512 non-null   object
 2   시군구코드   3512 non-null   int64 
 3   시군구명칭   3512 non-null   object
 4   읍면동코드   3512 non-null   int64 
 5   읍면동명칭   3512 non-null   object
dtypes: int64(3), object(3)
memory usage: 164.8+ KB


In [229]:
code['name'] = code.시도명칭.add(' ').add(code.시군구명칭)
code = code.drop_duplicates('name')
code.head()

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭,name
0,11,서울특별시,11010,종로구,1101053,사직동,서울특별시 종로구
17,11,서울특별시,11020,중구,1102052,소공동,서울특별시 중구
32,11,서울특별시,11030,용산구,1103051,후암동,서울특별시 용산구
48,11,서울특별시,11040,성동구,1104052,왕십리2동,서울특별시 성동구
65,11,서울특별시,11050,광진구,1105053,화양동,서울특별시 광진구


In [230]:
regions = list()
for triplog in df.trip.str.split(','):
    regions.extend(triplog)
regions = list(set(regions))

In [231]:
# regions : 쏘카 이용정보 triplog에 등장하는 unique한 지역들
regions = pd.DataFrame({'name':regions})
regions.head()

,name
0,경상북도 군위군
1,충청북도 충주시
2,서울특별시 양천구
3,서울특별시 서초구
4,서울특별시 금천구


In [232]:
regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    249 non-null    object
dtypes: object(1)
memory usage: 2.1+ KB


In [233]:
regions.shape

(249, 1)

In [234]:
regions = regions.merge(code[['name', '시군구코드']], how='left', on='name')
regions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    249 non-null    object 
 1   시군구코드   248 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.8+ KB


In [235]:
regions.shape

(249, 2)

In [236]:
regions[regions.시군구코드.isna()]

,name,시군구코드
40,세종특별자치시,NaN


In [237]:
regions.iloc[40, 1] = 29010

In [238]:
regions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    249 non-null    object 
 1   시군구코드   249 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.8+ KB


In [239]:
regions = regions.rename(columns={'시군구코드':'rcode'})

In [240]:
regions.rcode = regions.rcode.astype(int)

In [241]:
regions.head()

,name,rcode
0,경상북도 군위군,37310
1,충청북도 충주시,33020
2,서울특별시 양천구,11150
3,서울특별시 서초구,11220
4,서울특별시 금천구,11180


In [242]:
import os

path = '../data/region_data'

# foods = pd.read_csv(os.path.join(path, 'KC_618_LLR_RSTRT_CNBAS_TRND_2020.csv'))
stores = pd.read_csv(os.path.join(path, 'KC_619_DMSTC_TRV_CNSMP_STATN_BIZAEA_MAP_2019.csv'))
# tours = pd.read_csv(os.path.join(path, 'YN_REGNAL_TOUR_ND_TRNSPORT_GOODS_20200831.csv'))

In [243]:
stores.head()

,sido_nm,sgg_nm,hadm_cd,atrctn_cnt,rstrt_cnt,shopng_cnt,residnt_cnt_sum,FILE_NAME,base_ymd
0,강원도,강릉시,42150,308,4685,460,162905,KC_619_DMSTC_TRV_CNSMP_STATN_BIZAEA_MAP_2019,20200214
1,강원도,고성군,42820,105,795,117,18628,KC_619_DMSTC_TRV_CNSMP_STATN_BIZAEA_MAP_2019,20200214
2,강원도,동해시,42170,85,1929,247,68772,KC_619_DMSTC_TRV_CNSMP_STATN_BIZAEA_MAP_2019,20200214
3,강원도,삼척시,42230,135,1597,209,45409,KC_619_DMSTC_TRV_CNSMP_STATN_BIZAEA_MAP_2019,20200214
4,강원도,속초시,42210,79,2594,210,63203,KC_619_DMSTC_TRV_CNSMP_STATN_BIZAEA_MAP_2019,20200214


In [244]:
stores[stores.sgg_nm.isna()]

,sido_nm,sgg_nm,hadm_cd,atrctn_cnt,rstrt_cnt,shopng_cnt,residnt_cnt_sum,FILE_NAME,base_ymd
165,세종특별자치시,NaN,36110,152,3886,301,229563,KC_619_DMSTC_TRV_CNSMP_STATN_BIZAEA_MAP_2019,20200214


In [245]:
stores['name'] = stores.sido_nm + ' ' + stores.sgg_nm
stores = stores.drop(columns=['FILE_NAME', 'base_ymd', 'hadm_cd'])
stores = stores.merge(code[['name', '시군구코드']], how='left', on='name')

stores = stores.rename(columns={'시군구코드':'rcode'})

In [246]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sido_nm          250 non-null    object 
 1   sgg_nm           249 non-null    object 
 2   atrctn_cnt       250 non-null    int64  
 3   rstrt_cnt        250 non-null    int64  
 4   shopng_cnt       250 non-null    int64  
 5   residnt_cnt_sum  250 non-null    int64  
 6   name             249 non-null    object 
 7   rcode            249 non-null    float64
dtypes: float64(1), int64(4), object(3)
memory usage: 17.6+ KB


In [247]:
stores[stores.name.isna()].name = '세종특별자치시'
stores[stores.name.isna()].rcode = 29010

C:\Users\jomin\anaconda3\envs\socar37\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [248]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sido_nm          250 non-null    object 
 1   sgg_nm           249 non-null    object 
 2   atrctn_cnt       250 non-null    int64  
 3   rstrt_cnt        250 non-null    int64  
 4   shopng_cnt       250 non-null    int64  
 5   residnt_cnt_sum  250 non-null    int64  
 6   name             249 non-null    object 
 7   rcode            249 non-null    float64
dtypes: float64(1), int64(4), object(3)
memory usage: 17.6+ KB


In [249]:
regions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    249 non-null    object
 1   rcode   249 non-null    int32 
dtypes: int32(1), object(1)
memory usage: 4.9+ KB


In [250]:
stores.columns

Index(['sido_nm', 'sgg_nm', 'atrctn_cnt', 'rstrt_cnt', 'shopng_cnt',
       'residnt_cnt_sum', 'name', 'rcode'],
      dtype='object')

In [251]:
regions = regions.merge(stores[['atrctn_cnt', 'rstrt_cnt', 'shopng_cnt', 'residnt_cnt_sum', 'rcode']], how='left', on='rcode')

In [253]:
# triplog에 존재하는 모든 지역에 대한 특성정보를 추가함
# 이 테이블을 참조하여 이용정보에서 방문한 모든 지역의 특성을 조회할수있음
regions.head(10)

,name,rcode,atrctn_cnt,rstrt_cnt,shopng_cnt,residnt_cnt_sum
0,경상북도 군위군,37310,153.00,421.00,82.00,16253.00
1,충청북도 충주시,33020,260.00,4534.00,520.00,165765.00
2,서울특별시 양천구,11150,27.00,3716.00,432.00,385006.00
3,서울특별시 서초구,11220,87.00,6711.00,310.00,379715.00
4,서울특별시 금천구,11180,26.00,3305.00,301.00,244507.00
5,전라남도 함평군,36430,170.00,443.00,89.00,21135.00
6,부산광역시 동래구,21060,85.00,4146.00,320.00,214009.00
7,충청남도 서산시,34050,153.00,3201.00,311.00,131876.00
8,경기도 성남시 중원구,31022,30.00,2671.00,271.00,185399.00
9,경기도 용인시 수지구,31193,30.00,2254.00,218.00,286691.00
